In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        #if self.train_mode:
        #    augs = get_train_augs()
        #    img = augs(image=img)['image']
        
        img = np.expand_dims(img, axis=-1)
        #print('###', img.shape)
        img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

# model

In [12]:
#import pretrainedmodels

In [13]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [16]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

In [17]:
#model.last_linear.in_features

In [18]:
class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        #self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        #self.fix_input_layer()
        
    def fix_input_layer(self):
        if self.backbone_name in ['se_resnext50_32x4d', 'se_resnext101_32x4d', 'se_resnet50', 'senet154', 'se_resnet152', 'nasnetmobile', 'mobilenet', 'nasnetalarge']:
            #self.backbone = eval(backbone_name)()
            #print(self.backbone.layer0.conv1)
            w = self.backbone.layer0.conv1.weight.data
            self.backbone.layer0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            #self.backbone.layer0.conv1.weight = torch.nn.Parameter(torch.cat((w, w[:, 2, :, :].unsqueeze(1)), dim=1))
            self.backbone.layer0.conv1.weight = torch.nn.Parameter(w[:, 0, :, :].unsqueeze(1))
        
    def logits(self, x):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        #x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [19]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [20]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [21]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [22]:
round(1/9, 6)

0.111111

In [23]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [24]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [25]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [26]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [27]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [28]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [29]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [30]:
np.random.random()

0.2588587835461107

In [31]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam

In [32]:
def train(args):
    global model

    if args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
    elif args.optim == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    elif args.optim == 'Over9000':
        optimizer = Over9000(model.parameters(), lr=args.lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.)

    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.t_max, eta_min=args.min_lr)
        
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    #if args.lrs == 'plateau':
    #    lr_scheduler.step(best_metrics)
    #else:
    #    lr_scheduler.step()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #do_mixup = False #(np.random.random() < 0.4)
            
            #if do_mixup:
            #    img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            
            
            #if do_mixup:
            #    loss = mixup_criterion(outputs, targets)
            #else:
            #    loss = criterion(outputs, targets)
            r = np.random.rand()
            #if args.beta > 0 and r < args.cutmix_prob:
            if r < 0.5:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs = model(img)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
                img, targets = mixup(img, targets)
                outputs = model(img)
                loss = mixup_criterion(outputs, targets)
                #loss = criterion(outputs, targets)
            
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            
            #loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [33]:
args = Namespace()
args.backbone = 'se_resnext50_32x4d'
args.ckp_name = 'model3_se_resnext50_fold0_mixup_cutmix.pth'
args.predict = False
args.optim = 'Adam'
args.lr = 4e-5
args.lrs = 'cosine'
args.t_max = 12
args.factor = 0.1
args.patience = 0
args.min_lr = 1e-6
args.iter_val = 200
args.num_epochs = 100000
args.batch_size = 1024
args.val_batch_size = 1024

args.beta = 1.0
args.cutmix_prob = 0.5

In [34]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=0)

(200840, 5)
(200840, 32332)
(160596, 5) (40244, 5)


In [35]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


model file: ./models/se_resnext50_32x4d/model3_se_resnext50_fold0_mixup_cutmix.pth, exist: True
loading ./models/se_resnext50_32x4d/model3_se_resnext50_fold0_mixup_cutmix.pth...


In [ ]:
train(args)

{'recall': 0.984545, 'recall_grapheme': 0.977655, 'recall_vowel': 0.991144, 'recall_consonant': 0.991725, 'acc_grapheme': 0.976518, 'acc_vowel': 0.993067, 'acc_consonant': 0.991204, 'loss_grapheme': 0.173279, 'loss_vowel': 0.118327, 'loss_consonant': 0.087629}
    1 | 0.000040 | 045056/160596 | 1.1521 | 2.3504 |
val: {'recall': 0.98306, 'recall_grapheme': 0.974674, 'recall_vowel': 0.990755, 'recall_consonant': 0.992138, 'acc_grapheme': 0.975052, 'acc_vowel': 0.992744, 'acc_consonant': 0.991104, 'loss_grapheme': 0.199909, 'loss_vowel': 0.159334, 'loss_consonant': 0.112617}


/home/chec/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    2 | 0.000039 | 090112/160596 | 3.2593 | 2.0820 |
val: {'recall': 0.983107, 'recall_grapheme': 0.976135, 'recall_vowel': 0.990384, 'recall_consonant': 0.989775, 'acc_grapheme': 0.974903, 'acc_vowel': 0.992272, 'acc_consonant': 0.99093, 'loss_grapheme': 0.227628, 'loss_vowel': 0.170546, 'loss_consonant': 0.114405}
    3 | 0.000037 | 135168/160596 | 1.8368 | 2.2458 |
val: {'recall': 0.98319, 'recall_grapheme': 0.97552, 'recall_vowel': 0.989779, 'recall_consonant': 0.991943, 'acc_grapheme': 0.975947, 'acc_vowel': 0.992073, 'acc_consonant': 0.991055, 'loss_grapheme': 0.178566, 'loss_vowel': 0.130174, 'loss_consonant': 0.096297}
    5 | 0.000034 | 020480/160596 | 1.7801 | 2.2504 |
val: {'recall': 0.982983, 'recall_grapheme': 0.974734, 'recall_vowel': 0.991155, 'recall_consonant': 0.99131, 'acc_grapheme': 0.975574, 'acc_vowel': 0.992471, 'acc_consonant': 0.990955, 'loss_grapheme': 0.207346, 'loss_vowel': 0.135512, 'loss_consonant': 0.102086}
    6 | 0.000030 | 065536/160596 | 1.4792 | 2.1

   35 | 0.000034 | 143360/160596 | 2.4303 | 2.2059 |
val: {'recall': 0.983978, 'recall_grapheme': 0.976505, 'recall_vowel': 0.991757, 'recall_consonant': 0.991143, 'acc_grapheme': 0.976717, 'acc_vowel': 0.993067, 'acc_consonant': 0.991552, 'loss_grapheme': 0.211782, 'loss_vowel': 0.154665, 'loss_consonant': 0.110613}
   37 | 0.000030 | 028672/160596 | 3.3289 | 2.2846 |
val: {'recall': 0.983313, 'recall_grapheme': 0.975249, 'recall_vowel': 0.990985, 'recall_consonant': 0.991769, 'acc_grapheme': 0.975226, 'acc_vowel': 0.992645, 'acc_consonant': 0.991328, 'loss_grapheme': 0.221947, 'loss_vowel': 0.16989, 'loss_consonant': 0.116109}
   38 | 0.000026 | 073728/160596 | 2.3663 | 2.0064 |
val: {'recall': 0.983726, 'recall_grapheme': 0.975903, 'recall_vowel': 0.991326, 'recall_consonant': 0.991774, 'acc_grapheme': 0.97617, 'acc_vowel': 0.993067, 'acc_consonant': 0.991527, 'loss_grapheme': 0.195528, 'loss_vowel': 0.148875, 'loss_consonant': 0.099527}
   39 | 0.000021 | 118784/160596 | 3.7305 | 2

** saved
   70 | 0.000021 | 081920/160596 | 2.3546 | 2.2292 |
val: {'recall': 0.984699, 'recall_grapheme': 0.977967, 'recall_vowel': 0.991848, 'recall_consonant': 0.991015, 'acc_grapheme': 0.976792, 'acc_vowel': 0.992893, 'acc_consonant': 0.99175, 'loss_grapheme': 0.238455, 'loss_vowel': 0.189144, 'loss_consonant': 0.130037}
   71 | 0.000015 | 126976/160596 | 4.1406 | 2.1631 |
val: {'recall': 0.984664, 'recall_grapheme': 0.977666, 'recall_vowel': 0.992009, 'recall_consonant': 0.991316, 'acc_grapheme': 0.976469, 'acc_vowel': 0.99339, 'acc_consonant': 0.991701, 'loss_grapheme': 0.217698, 'loss_vowel': 0.167465, 'loss_consonant': 0.115071}
   73 | 0.000011 | 012288/160596 | 1.4217 | 1.9212 |
val: {'recall': 0.984486, 'recall_grapheme': 0.976901, 'recall_vowel': 0.992169, 'recall_consonant': 0.991972, 'acc_grapheme': 0.977115, 'acc_vowel': 0.993291, 'acc_consonant': 0.991651, 'loss_grapheme': 0.184145, 'loss_vowel': 0.136103, 'loss_consonant': 0.097535}
   74 | 0.000007 | 057344/160596 | 1

  103 | 0.000011 | 135168/160596 | 1.9478 | 2.0246 |
val: {'recall': 0.984892, 'recall_grapheme': 0.977473, 'recall_vowel': 0.991793, 'recall_consonant': 0.992828, 'acc_grapheme': 0.977388, 'acc_vowel': 0.993365, 'acc_consonant': 0.991974, 'loss_grapheme': 0.167989, 'loss_vowel': 0.119791, 'loss_consonant': 0.092011}
  105 | 0.000007 | 020480/160596 | 3.5860 | 2.2800 |
val: {'recall': 0.984914, 'recall_grapheme': 0.977796, 'recall_vowel': 0.991765, 'recall_consonant': 0.992297, 'acc_grapheme': 0.977214, 'acc_vowel': 0.993564, 'acc_consonant': 0.992123, 'loss_grapheme': 0.212985, 'loss_vowel': 0.170657, 'loss_consonant': 0.118314}
  106 | 0.000004 | 065536/160596 | 1.3849 | 1.9246 |
val: {'recall': 0.985031, 'recall_grapheme': 0.978166, 'recall_vowel': 0.991633, 'recall_consonant': 0.992159, 'acc_grapheme': 0.977388, 'acc_vowel': 0.993341, 'acc_consonant': 0.99185, 'loss_grapheme': 0.163974, 'loss_vowel': 0.117736, 'loss_consonant': 0.087004}
  107 | 0.000002 | 110592/160596 | 0.8148 | 

  137 | 0.000004 | 028672/160596 | 2.2479 | 1.9912 |
val: {'recall': 0.984948, 'recall_grapheme': 0.977878, 'recall_vowel': 0.992624, 'recall_consonant': 0.991412, 'acc_grapheme': 0.978282, 'acc_vowel': 0.993862, 'acc_consonant': 0.992073, 'loss_grapheme': 0.190926, 'loss_vowel': 0.146448, 'loss_consonant': 0.104501}
  138 | 0.000002 | 073728/160596 | 1.6229 | 2.1266 |
val: {'recall': 0.985144, 'recall_grapheme': 0.978284, 'recall_vowel': 0.992314, 'recall_consonant': 0.991693, 'acc_grapheme': 0.978009, 'acc_vowel': 0.993763, 'acc_consonant': 0.992148, 'loss_grapheme': 0.204986, 'loss_vowel': 0.167922, 'loss_consonant': 0.114289}
  139 | 0.000001 | 118784/160596 | 2.4991 | 2.0289 |
val: {'recall': 0.984906, 'recall_grapheme': 0.977896, 'recall_vowel': 0.992209, 'recall_consonant': 0.991623, 'acc_grapheme': 0.97786, 'acc_vowel': 0.993838, 'acc_consonant': 0.992148, 'loss_grapheme': 0.194758, 'loss_vowel': 0.147456, 'loss_consonant': 0.106529}
  141 | 0.000002 | 004096/160596 | 0.2932 | 

  170 | 0.000001 | 081920/160596 | 0.9312 | 1.9516 |
val: {'recall': 0.985514, 'recall_grapheme': 0.978801, 'recall_vowel': 0.992703, 'recall_consonant': 0.99175, 'acc_grapheme': 0.978432, 'acc_vowel': 0.993862, 'acc_consonant': 0.992496, 'loss_grapheme': 0.190729, 'loss_vowel': 0.138661, 'loss_consonant': 0.099952}
  171 | 0.000002 | 126976/160596 | 1.6063 | 2.0181 |
val: {'recall': 0.985266, 'recall_grapheme': 0.978077, 'recall_vowel': 0.992169, 'recall_consonant': 0.99274, 'acc_grapheme': 0.978307, 'acc_vowel': 0.993589, 'acc_consonant': 0.992396, 'loss_grapheme': 0.19008, 'loss_vowel': 0.138038, 'loss_consonant': 0.102743}
  173 | 0.000004 | 012288/160596 | 1.6167 | 1.8774 |
val: {'recall': 0.98541, 'recall_grapheme': 0.978169, 'recall_vowel': 0.992652, 'recall_consonant': 0.992651, 'acc_grapheme': 0.978258, 'acc_vowel': 0.993763, 'acc_consonant': 0.992222, 'loss_grapheme': 0.168332, 'loss_vowel': 0.115569, 'loss_consonant': 0.088658}
  174 | 0.000007 | 057344/160596 | 2.8782 | 2.0

** saved
  203 | 0.000004 | 135168/160596 | 4.1408 | 2.0976 |
val: {'recall': 0.985651, 'recall_grapheme': 0.979445, 'recall_vowel': 0.992179, 'recall_consonant': 0.991536, 'acc_grapheme': 0.978829, 'acc_vowel': 0.994036, 'acc_consonant': 0.99257, 'loss_grapheme': 0.220154, 'loss_vowel': 0.174584, 'loss_consonant': 0.125366}
  205 | 0.000007 | 020480/160596 | 2.6845 | 2.1228 |
val: {'recall': 0.985931, 'recall_grapheme': 0.979909, 'recall_vowel': 0.99274, 'recall_consonant': 0.991165, 'acc_grapheme': 0.978829, 'acc_vowel': 0.994136, 'acc_consonant': 0.992595, 'loss_grapheme': 0.208803, 'loss_vowel': 0.164151, 'loss_consonant': 0.114046}
  206 | 0.000011 | 065536/160596 | 0.9440 | 2.0346 |
val: {'recall': 0.986478, 'recall_grapheme': 0.980274, 'recall_vowel': 0.992748, 'recall_consonant': 0.992615, 'acc_grapheme': 0.979028, 'acc_vowel': 0.993962, 'acc_consonant': 0.992372, 'loss_grapheme': 0.18946, 'loss_vowel': 0.145447, 'loss_consonant': 0.104203}
** saved
  207 | 0.000015 | 110592/16

In [ ]:
train(args)

{'recall': 0.978358, 'recall_grapheme': 0.967606, 'recall_vowel': 0.987796, 'recall_consonant': 0.990422, 'acc_grapheme': 0.967374, 'acc_vowel': 0.989241, 'acc_consonant': 0.987998, 'loss_grapheme': 0.33989, 'loss_vowel': 0.214178, 'loss_consonant': 0.145039}
    1 | 0.000040 | 045056/160596 | 4.0782 | 3.0389 |
val: {'recall': 0.97827, 'recall_grapheme': 0.967657, 'recall_vowel': 0.987124, 'recall_consonant': 0.99064, 'acc_grapheme': 0.967225, 'acc_vowel': 0.988918, 'acc_consonant': 0.987576, 'loss_grapheme': 0.370762, 'loss_vowel': 0.25688, 'loss_consonant': 0.164513}
    2 | 0.000039 | 090112/160596 | 5.0198 | 2.5940 |
val: {'recall': 0.976955, 'recall_grapheme': 0.965596, 'recall_vowel': 0.988205, 'recall_consonant': 0.988424, 'acc_grapheme': 0.966927, 'acc_vowel': 0.989241, 'acc_consonant': 0.987973, 'loss_grapheme': 0.266497, 'loss_vowel': 0.173287, 'loss_consonant': 0.117974}
    3 | 0.000037 | 135168/160596 | 0.2808 | 2.7450 |
val: {'recall': 0.978897, 'recall_grapheme': 0.96761

   33 | 0.000039 | 053248/160596 | 1.0196 | 2.7549 |
val: {'recall': 0.979197, 'recall_grapheme': 0.968548, 'recall_vowel': 0.989231, 'recall_consonant': 0.990463, 'acc_grapheme': 0.970232, 'acc_vowel': 0.990558, 'acc_consonant': 0.989216, 'loss_grapheme': 0.269461, 'loss_vowel': 0.194371, 'loss_consonant': 0.132939}
   34 | 0.000037 | 098304/160596 | 1.2563 | 2.5426 |
val: {'recall': 0.979079, 'recall_grapheme': 0.968473, 'recall_vowel': 0.989393, 'recall_consonant': 0.989979, 'acc_grapheme': 0.96961, 'acc_vowel': 0.990185, 'acc_consonant': 0.988868, 'loss_grapheme': 0.277417, 'loss_vowel': 0.199627, 'loss_consonant': 0.146769}
   35 | 0.000034 | 143360/160596 | 1.6984 | 2.5205 |
val: {'recall': 0.978954, 'recall_grapheme': 0.969456, 'recall_vowel': 0.988736, 'recall_consonant': 0.988169, 'acc_grapheme': 0.970082, 'acc_vowel': 0.990061, 'acc_consonant': 0.988619, 'loss_grapheme': 0.258687, 'loss_vowel': 0.163272, 'loss_consonant': 0.114384}
   37 | 0.000030 | 028672/160596 | 3.7338 | 

   66 | 0.000034 | 106496/160596 | 4.7684 | 2.3626 |
val: {'recall': 0.980162, 'recall_grapheme': 0.971361, 'recall_vowel': 0.98908, 'recall_consonant': 0.988848, 'acc_grapheme': 0.97217, 'acc_vowel': 0.991477, 'acc_consonant': 0.989265, 'loss_grapheme': 0.241574, 'loss_vowel': 0.176307, 'loss_consonant': 0.122269}
   67 | 0.000030 | 151552/160596 | 0.2235 | 2.5852 |
val: {'recall': 0.9805, 'recall_grapheme': 0.971908, 'recall_vowel': 0.989568, 'recall_consonant': 0.988616, 'acc_grapheme': 0.972369, 'acc_vowel': 0.99175, 'acc_consonant': 0.989589, 'loss_grapheme': 0.236197, 'loss_vowel': 0.168501, 'loss_consonant': 0.116279}
   69 | 0.000026 | 036864/160596 | 3.6408 | 2.5842 |
val: {'recall': 0.980308, 'recall_grapheme': 0.971174, 'recall_vowel': 0.989555, 'recall_consonant': 0.989331, 'acc_grapheme': 0.971847, 'acc_vowel': 0.99175, 'acc_consonant': 0.989986, 'loss_grapheme': 0.258804, 'loss_vowel': 0.188044, 'loss_consonant': 0.133333}
   70 | 0.000021 | 081920/160596 | 1.7791 | 2.539

   99 | 0.000026 | 159744/160596 | 2.7826 | 2.4088 |
val: {'recall': 0.981572, 'recall_grapheme': 0.972929, 'recall_vowel': 0.990296, 'recall_consonant': 0.990135, 'acc_grapheme': 0.973338, 'acc_vowel': 0.992024, 'acc_consonant': 0.990284, 'loss_grapheme': 0.26153, 'loss_vowel': 0.198119, 'loss_consonant': 0.149081}
  101 | 0.000021 | 045056/160596 | 0.9083 | 2.5024 |
val: {'recall': 0.982115, 'recall_grapheme': 0.973911, 'recall_vowel': 0.990267, 'recall_consonant': 0.990374, 'acc_grapheme': 0.974158, 'acc_vowel': 0.992073, 'acc_consonant': 0.99021, 'loss_grapheme': 0.232243, 'loss_vowel': 0.167849, 'loss_consonant': 0.117313}
  102 | 0.000015 | 090112/160596 | 3.7902 | 2.4828 |
val: {'recall': 0.982729, 'recall_grapheme': 0.975183, 'recall_vowel': 0.989947, 'recall_consonant': 0.990602, 'acc_grapheme': 0.975052, 'acc_vowel': 0.991899, 'acc_consonant': 0.990309, 'loss_grapheme': 0.283059, 'loss_vowel': 0.20083, 'loss_consonant': 0.142793}
** saved
  103 | 0.000011 | 135168/160596 | 2.

  133 | 0.000015 | 053248/160596 | 1.1055 | 2.3244 |
val: {'recall': 0.982106, 'recall_grapheme': 0.973416, 'recall_vowel': 0.990781, 'recall_consonant': 0.990809, 'acc_grapheme': 0.974729, 'acc_vowel': 0.992421, 'acc_consonant': 0.991253, 'loss_grapheme': 0.211503, 'loss_vowel': 0.166163, 'loss_consonant': 0.111303}
  134 | 0.000011 | 098304/160596 | 2.0396 | 2.2117 |
val: {'recall': 0.982572, 'recall_grapheme': 0.974751, 'recall_vowel': 0.990227, 'recall_consonant': 0.990558, 'acc_grapheme': 0.975326, 'acc_vowel': 0.992297, 'acc_consonant': 0.991154, 'loss_grapheme': 0.219274, 'loss_vowel': 0.157781, 'loss_consonant': 0.109228}
  135 | 0.000007 | 143360/160596 | 3.6337 | 2.3152 |
val: {'recall': 0.982873, 'recall_grapheme': 0.974579, 'recall_vowel': 0.990673, 'recall_consonant': 0.991662, 'acc_grapheme': 0.975027, 'acc_vowel': 0.992446, 'acc_consonant': 0.99098, 'loss_grapheme': 0.202821, 'loss_vowel': 0.147704, 'loss_consonant': 0.105511}
  137 | 0.000004 | 028672/160596 | 2.3300 | 

  165 | 0.000007 | 130048/160596 | 0.0454 | 2.0883 |

In [ ]:
#save_model(model, model_file)

In [42]:
del model